In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
from nltk.stem import WordNetLemmatizer 
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
from nltk import corpus
from nltk import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
import os
import re

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install pyLDAvis 

In [ ]:
#data loader class
class CsvLoader:
  def __init__(self,path,filename):
    if os.getcwd() != path:
      os.chdir(path)   
    self.path = path
    self.filename = filename 
 
  def load_csv(self):
    df = pd.read_csv(self.filename,engine='python',error_bad_lines=False)
    return df
  

In [ ]:
csv_loader = CsvLoader('/content','processed_tweet_data.csv')
csv_loaded = csv_loader.load_csv()
csv_loaded.dropna(subset = ["original_text"], inplace=True)
csv_loaded

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Fri Jun 18 17:55:49 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548.0,612.0,ketuesriche,551,351,NaN,NaN,TelGlobalHealthWHOAFRO,Mass
1,Fri Jun 18 17:55:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195.0,92.0,Grid1949,66,92,NaN,NaN,globalhlthtwit,"Edinburgh, Scotland"
2,Fri Jun 18 17:56:07 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Thank you @research2note for creating this ama...,0.316667,0.483333,en,2.0,1.0,LeeTomlinson8,1195,1176,NaN,red4research,NHSRDForumResearch2noteNHSRDForum,NaN
3,Fri Jun 18 17:56:10 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1580.0,899.0,RIPNY08,2666,2704,NaN,NaN,HighWireTalk,NaN
4,Fri Jun 18 17:56:20 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",I think it’s important that we don’t sell COVA...,0.280000,0.620000,en,72.0,20.0,pash22,28250,30819,NaN,NaN,PeterHotez,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6521,Sat Jun 19 07:40:29 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Australia is sending vaccines.\nAustralia is s...,0.100000,0.400000,en,78.0,12.0,ravaifiji,1577,538,NaN,NaN,pseudofijian,Fiji
6522,Sat Jun 19 07:40:29 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...",The Truth Behind COVID-19 Vaccines (6) —— Bell...,-0.386111,0.622222,en,2.0,2.0,Hhuatakedownccp,697,303,NaN,DrLiMengYanCCPVirus,edendomain,"Osaka-shi Chuo, Osaka"
6524,Sat Jun 19 07:41:00 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",Covid19 vaccines reach the remotest places of ...,-0.050000,0.400000,en,113.0,87.0,bitu_50,410,483,NaN,NaN,N_RamchanderRao,"Jehanabad, India"
6528,Sat Jun 19 07:41:26 +0000 2021,"<a href=""http://twitter.com/download/android"" ...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1595.0,908.0,wayno_af007,2224,2739,NaN,NaN,HighWireTalk,"The boro, MA"


#Data Understanding
Display information on the Data

In [ ]:
class DataInfo:
  def __init__(self,df):
    self.df = df

  def general_info(self):
    return self.df.info()

In [ ]:
Get_Info = DataInfo(csv_loaded)
Get_Info.general_info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3720 entries, 0 to 6530
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   created_at          3720 non-null   object 
 1   source              3720 non-null   object 
 2   original_text       3720 non-null   object 
 3   polarity            3720 non-null   float64
 4   subjectivity        3720 non-null   float64
 5   lang                3720 non-null   object 
 6   favorite_count      3720 non-null   float64
 7   retweet_count       3720 non-null   float64
 8   original_author     3720 non-null   object 
 9   followers_count     3720 non-null   int64  
 10  friends_count       3720 non-null   int64  
 11  possibly_sensitive  47 non-null     object 
 12  hashtags            2060 non-null   object 
 13  user_mentions       3720 non-null   object 
 14  place               2478 non-null   object 
dtypes: float64(4), int64(2), object(9)
memory usage: 465.0+

In [ ]:

  # def remove_symbols(self,sentence_list):
  #    regrex_pattern = re.compile(pattern = "["
  #       u"\U0001F600-\U0001F64F"  # emoticons
  #       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
  #       u"\U0001F680-\U0001F6FF"  # transport & map symbols
  #       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
  #                          "]+", flags = re.UNICODE)
  #    return regrex_pattern.sub(r'',sentence_list)


#Data Preparation
Select and clean the data we are going to use for our model

In [244]:
class PreparedData:
  def __init__(self,df):
    self.df = df
      
  def clean_df(self):
    df = self.df[self.df['lang'] =="en"]
    df['original_text'] = df['original_text'].astype(str)
    df['original_text'] = df['original_text'].apply(lambda x: x.lower())
    
    return df

  def select_data(self,df):
    sel_df = self.df['original_text']
    sentence_list = sel_df.tolist()
    return sentence_list

  def tokenize_data(self,sentence_list):
    #creating sentences and words from the input tweets
    words = []
    tokenizer1 = RegexpTokenizer("[a-zA-Z]+")
    for sent in sentence_list:
        for tokenized in tokenizer1.tokenize(sent):
          words.append(tokenized)   
    return words 

  def lemmatize_data(self,word):
    wl = WordNetLemmatizer()
    return str(wl.lemmatize(word))

  def remove_stop_words(self,word):
    s_words = stopwords.words('english')
    if word not in s_words:
      return word

  def remove_none(self,wordlist):
    cleaned_list = []
    for w in wordlist:
      if w is not None:
        cleaned_list.append(w)
    return cleaned_list
  
  

In [255]:
def prepare_data(df):
    PreparedData_object = PreparedData(df)
    cleaned_df = PreparedData_object.clean_df()
    selected = PreparedData_object.select_data(cleaned_df)
    word_list = PreparedData_object.tokenize_data(selected)
    prepared_wordList = [PreparedData_object.remove_stop_words(PreparedData_object.lemmatize_data(word)) for word in word_list]
    prepared = PreparedData_object.remove_none(prepared_wordList)
    return prepared,selected
    

    

In [256]:
prepared_wordlist,prepared_sentencelist = prepare_data(csv_loaded)

In [261]:
prepared_wordlist[0]

'Africa'

#Modeling
Selecting the right model and assessing it 

In [ ]:
dictionary= []
word_id = corpora.Dictionary(prepared_wordlist)
for word in prepared_wordlist:
  dictionary.append(word_to_id.doc2bow(word))

In [ ]:
# Modeling
model = gensim.models.ldamodel.LdaModel(dictionary,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Evaluation using coherence
lda_coherence_mod = CoherenceModel(model=model, texts=words, dictionary=id2word, coherence='c_v')
coherence = lda_coherence_mod.get_coherence()
print(f'Coherence score and accuracy {coherence}', )

Coherence score and accuracy 0.3790233056254021


In [ ]:
pprint(model.show_topics(formatted=False))

[(0,
  [('to', 0.04990605),
   ('are', 0.030841904),
   ('the', 0.028928673),
   ('covid19', 0.026192188),
   ('vaccination', 0.021781636),
   ('vaccine', 0.019142067),
   ('there', 0.01649909),
   ('today', 0.014926311),
   ('vaccines', 0.011860952),
   ('a', 0.011256056)]),
 (1,
  [('to', 0.04598122),
   ('covid19', 0.027335439),
   ('vaccines', 0.025929095),
   ('the', 0.025254969),
   ('a', 0.015656063),
   ('is', 0.014528551),
   ('and', 0.014212991),
   ('amp', 0.013651507),
   ('need', 0.013451699),
   ('we', 0.012701644)]),
 (2,
  [('the', 0.044056974),
   ('india', 0.031787246),
   ('of', 0.03163889),
   ('and', 0.028904216),
   ('covid19', 0.023269067),
   ('in', 0.023034072),
   ('vaccines', 0.020947216),
   ('to', 0.019840507),
   ('a', 0.015743049),
   ('can', 0.013278673)]),
 (3,
  [('the', 0.05347785),
   ('of', 0.03792886),
   ('in', 0.03293381),
   ('covid19', 0.022712208),
   ('vaccines', 0.018106144),
   ('to', 0.017966766),
   ('doses', 0.014813065),
   ('been', 0.0

Visualization

In [ ]:
#Visualizing
pyLDAvis.enable_notebook()
LDAvis_prepared = gensimvis.prepare(model, dictionary, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.130733  0.045423       1        1  28.101745
1      0.055109  0.226078       2        1  23.851692
2     -0.066829  0.004994       3        1  17.554202
0      0.297442 -0.108393       4        1  15.592869
4     -0.154988 -0.168101       5        1  14.899492, topic_info=         Term         Freq        Total Category  logprob  loglift
266     india   810.000000   810.000000  Default  30.0000  30.0000
34     africa  1348.000000  1348.000000  Default  29.0000  29.0000
22         of  3576.000000  3576.000000  Default  28.0000  28.0000
5         and  2511.000000  2511.000000  Default  27.0000  27.0000
16         in  3090.000000  3090.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
62   vaccines   221.132478  2658.216426   Topic5  -4.5824  -0.5828
50        one   163.005332   458.926986   Topic5  -4.8874   0.8687
40        for   174.851201  1116.876304   Topic5  -4.8172   0.0495
42     health   159.160015   535.948102   Topic5  -4.9112   0.6897
37         dr   150.658186   390.593513   Topic5  -4.9661   0.9512

[311 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
154       4  0.996996  18002320233
280       1  0.995523            2
1         2  0.080234           20
1         5  0.914667           20
4728      4  0.993849       204344
...     ...       ...          ...
177       4  0.992426            📞
179       4  0.994144            🔍
5654      3  0.992537            🔰
32        5  0.988932      🚨africa
33        5  0.994762            🧵

[384 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1, 5])